In [1]:
!cd

D:\Ansys Simulations\Project\2D\preprocessing


## Formatting
In this notebook, the objective is to explore and develop a way to transformed the scaled data into a relevant data format/file/shape that can be used for a 2d convolutional neural network. Ideally, what is desired is a file that can store tensors as spatial data and that can be used by a ConvNet.